# Google Earth Engine- Plotting Fire Data on a Bubble Map 

My name is Winifred, and I am a second-year student studying Computer Science at UC Berkeley. I was first exposed to data science one year ago and quickly realized just how applicable it is to various fields of study. After joining the Cabinet of Curiosity team, I learned more about existing natural history databases and collaborated with other team members to dive into and study these databases. I became particularly interested in the [Google Earth Engine database](https://earthengine.google.com/) and the vast amount of geophysical, climate, and imagery data it contained. Data visualization was also something I wanted to learn more about, which led me to exploring California wildfire data from the Earth Engine database and using Plotly to create a map that represented this data.. 

## Acquiring Data From Google Earth Engine

Google Earth Engine hosts satellite imagery and other geospatial data and is a tool that allows for analysis of these datasets, which are made available for global-scale data mining. Images of Earth may go back more than 40 years and are ingested on a daily basis. Datasets include the USGS/NASA Landsat catalog, precipitation data, elevation data, and many more. Earth Engine also allows users to upload their own data for analysis. 

Earth Engine has been used in many interesting case studies, such as the [Map of Life](http://species.mol.org/species/map/Perdix_dauurica), as well as analysis of [global forest cover change](http://species.mol.org/species/map/Perdix_dauurica).

## Downloading Data 

Browse the [Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/) to find a dataset that interests you. I picked one that contains [data about fires](https://developers.google.com/earth-engine/datasets/catalog/FIRMS). Notice the link  listed under the "Dataset Provider" section.

<img src = "../images/dataset_provider.PNG">


FIRMS provides "active fire data", which consists of data from the past 24 hours, 48 hours, and 7 days. However, because FIRMS data dates back to November 2000, I wanted to know how to download data older than 7 days.

Once you've reached the FIRMS dataset provider link, scroll down and select "Archive Download." Then, click on "Create New Request." 

<img src='../images/nasa_page.PNG'>

<img src='../images/archive_page.png'>

I wanted to narrow down the data to Southern California wildfires, so in the Download Request box, I decided to get data on a custom region and drew a polygon that roughly enclosed California. For fire data source, I chose VIIRS, which is an instrument  used to detect fires and gather data. My date range of choice was September 1st 2017 to September 1st 2018, and I decided to download it as a .csv file. Finally, you must enter your email so that FIRMS can send the data to you. You should receive an email within a few days.

<img src='../images/download_request.png'>

Once you receive the email, just download the .csv file, and you're ready to go! 

## Using Plotly for Python 

[Plotly](https://plot.ly/#/) has a [Python library](https://plot.ly/python/) that allows you to create interactive data visualizations online, such as line plots, scatter maps, heatmaps, and 3D network graphs. In this tutorial, we will create a bubble map that visualizes wildfire data in California (based on the FIRMS data we acquired). 

Follow the [Getting Started](https://plot.ly/python/getting-started/) tutorial to set up the Plotly.py library. Once that is done, let's import some libraries that we will use in this tutorial:

In [1]:
import plotly.plotly as py
from datascience import *
import numpy as np

Our FIRMS dataset is quite big, so let's narrow it down to just a few points on the map. I'd like to plot the 5 brightest, 5 middle, and 5 dimmest wildfires in the dataset. 

First, let's load the .csv file into a table:

In [2]:
data=Table().read_table('~/URAP/google_earth_fires/data/SoCal_fires.csv') #from https://github.com/jupyterhub/jupyterhub/issues/376
data

latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,type
36.234,-118.43,321.9,1,1,2017-10-07,1845,Terra,MODIS,44,6.1,302.5,14.5,0
36.258,-118.421,319.4,4.7,2,2017-10-15,1756,Terra,MODIS,72,6.1,294.9,138.7,0
36.259,-118.425,318.2,4.7,2,2017-10-15,1756,Terra,MODIS,44,6.1,294.4,118.3,0
36.256,-118.42,345.1,1.1,1,2017-10-15,2112,Aqua,MODIS,92,6.1,298.3,50,0
36.272,-118.429,326.3,1.1,1,2017-10-16,1839,Terra,MODIS,71,6.1,293,23.1,0
36.271,-118.418,331.1,1.1,1,2017-10-16,1839,Terra,MODIS,83,6.1,292.7,29.9,0
36.255,-118.412,300,3,1.6,2017-10-16,913,Aqua,MODIS,5,6.1,281.5,31.6,0
36.256,-118.428,327,4,1.9,2017-10-16,2017,Aqua,MODIS,77,6.1,278.8,192.8,0
36.253,-118.428,330.3,1.4,1.2,2017-10-17,542,Terra,MODIS,100,6.1,281.1,49.7,0
36.412,-119.232,339.2,1.7,1.3,2017-09-08,1916,Terra,MODIS,87,6.1,302.5,73.2,0


## Filtering the data

Now, let's select our columns of interest. For the purposes of this tutorial, we care about the latitude, longitude, brightness, and dates of our data.

In [3]:
py_data=data.select(0,1,2,5) #get the columns you want
py_data

latitude,longitude,brightness,acq_date
36.234,-118.43,321.9,2017-10-07
36.258,-118.421,319.4,2017-10-15
36.259,-118.425,318.2,2017-10-15
36.256,-118.42,345.1,2017-10-15
36.272,-118.429,326.3,2017-10-16
36.271,-118.418,331.1,2017-10-16
36.255,-118.412,300,2017-10-16
36.256,-118.428,327,2017-10-16
36.253,-118.428,330.3,2017-10-17
36.412,-119.232,339.2,2017-09-08


Next, let's find the top 5 brightest fires in our data. To do this, we can sort the brightness from brightest to dimmest, and use the numpy.take method to grab the first 5 rows. You will also do the same thing to find the 5 dimmest fires, except you will sort from dimmest to brightest.

In [4]:
brightest=py_data.sort('brightness', descending=True).take(range(0,5))
brightest

latitude,longitude,brightness,acq_date
34.466,-119.078,504,2017-12-06
34.47,-119.652,503.7,2017-12-16
34.549,-119.35,502.4,2017-12-11
34.415,-119.41,502.3,2017-12-10
34.477,-119.079,501.9,2017-12-06


In [5]:
dimmest=py_data.sort('brightness',descending=False).take(range(0,5))
dimmest

latitude,longitude,brightness,acq_date
36.255,-118.412,300,2017-10-16
34.339,-118.32,300,2017-12-06
37.681,-119.592,300,2017-10-08
37.212,-119.418,300,2017-12-06
36.693,-119.7,300,2018-01-26


Finding the middle 5 fires will take a bit more work. First, we will find the median of our brightness values. Then, we will find the index of table in which the median is located. Finally, we will want to take the two rows above the median and the two rows below the median to get the middle 5 of the table. 

In [6]:
bright_col=py_data.sort('brightness',descending=False).column('brightness') #the brightness column of our data
medium=np.median(bright_col) #find median of brightnesses
list(bright_col).index(medium) #find index of median


3017

In [7]:
middle=py_data.take(range((3017-2),(3017+3))) #take two above and two below median to get "middle 5 brightest"
middle

latitude,longitude,brightness,acq_date
34.456,-119.371,300.1,2017-12-09
34.478,-119.409,311.4,2017-12-09
34.476,-119.421,313.2,2017-12-09
34.474,-119.431,313.6,2017-12-09
34.436,-119.351,367.4,2017-12-09


Now, it's time to concatenate all our data! I am going to create arrays for each variable that we are interested in, using data from our "brightest", "middle", and "dimmest" tables.

In [8]:
lats=np.append(np.append(brightest.column(0),(middle.column(0))), dimmest.column(0)) #latitudes  
longs=np.append(np.append(brightest.column(1),(middle.column(1))), dimmest.column(1)) #longitudes
brights=np.append(np.append(brightest.column(2),(middle.column(2))), dimmest.column(2)) #brightnesses
dates=np.append(np.append(brightest.column(3),(middle.column(3))), dimmest.column(3)) #dates

Now, let's create a new table with columns for these variables! This table will contain the latitude, longitude, brightness, and dates of the top 5 brightest, middle 5, and 5 dimmest fire data. 

In [9]:
filtered_data=Table().with_columns(
        'latitude', lats,
        'longitude', longs,
        'brightness', brights,
        'acq_date', dates
        )
filtered_data

latitude,longitude,brightness,acq_date
34.466,-119.078,504,2017-12-06
34.47,-119.652,503.7,2017-12-16
34.549,-119.35,502.4,2017-12-11
34.415,-119.41,502.3,2017-12-10
34.477,-119.079,501.9,2017-12-06
34.456,-119.371,300.1,2017-12-09
34.478,-119.409,311.4,2017-12-09
34.476,-119.421,313.2,2017-12-09
34.474,-119.431,313.6,2017-12-09
34.436,-119.351,367.4,2017-12-09


## Plotting the data

Here's where things get a bit tricky. Ultimately, we would like to plot a figure with a specific set of data and a specific layout. Plotly has many attributes to help us accomplish our goal, but how do we know which ones to use? Luckily, Plotly has a [figure reference!](https://plot.ly/python/reference/)

Let's initialize our data first. We want to have different-colored bubbles on our bubble map so that we can differentiate between the three levels of brightness. Let's create a list of colors to choose from, using their RGB values: 

In [10]:
colors = ["rgb(255,0,0)", "rgb(255,128,0)", "rgb(255,255,0)"] #red, orange, yellow

Next, we'll create a list called limits. Limits contains three elements, and each element contains the beginning index and ending index of a brightness level. This list will allow us to map certain attributes to a specific range of data. For example, if we assign the first element of [colors] to the first element in [limits], the 5 brightest fires will appear as red bubbles on the map.

In [11]:
limits=[(0,5),(5,10),(10,15)] #we are grouping by brightness, where first five (0,5) are brightest, second group of five (5,10) are medium, etc.

Now, it's time to create a list that contains all the information that we want to represent on the map. We'll call this list "fires." In the end, we want this list to hold three dictionaries (one for each brightness level). 

Since we have three brightness levels, we want to assign specific attributes to each of them. To do this, we will iterature through each of the groups and create a dictionary of attributes for each. If you are unclear about these attributes, refer to the [figure reference!](https://plot.ly/python/reference/).

In [12]:
fires=[] #the data that we want to represent on the map
for i in range(len(limits)): #we want to iterate 3 times to create 3 traces on the map 
    group=filtered_data.take(range(limits[i][0], limits[i][1])) #i.e: in the first iteration, we are taking the top 5 rows (the brightest) 
    fire=dict(
        type = 'scattergeo', #the type of figure we want to create 
        locationmode = 'USA-states', 
        lon = group['longitude'],
        lat = group['latitude'],
        marker=dict(
                size=group['brightness'].item(0)/20, #scale the size of the bubble; our bubble size is based on the brightness 
                color=colors[i], #the color of the bubbles in this group 
                line = dict(width=0.5, color='rgb(40,40,40)'), 
                sizemode='diameter'
                ), #sets the properties of the bubbles 
        name='{0}-{1}'.format(limits[i][0], limits[i][1])) #legend labels
    fires.append(fire)

Let's see what our "fires" dictionary looks like: 

In [13]:
fires

[{'lat': array([34.466, 34.47 , 34.549, 34.415, 34.477]),
  'locationmode': 'USA-states',
  'lon': array([-119.078, -119.652, -119.35 , -119.41 , -119.079]),
  'marker': {'color': 'rgb(255,0,0)',
   'line': {'color': 'rgb(40,40,40)', 'width': 0.5},
   'size': 25.2,
   'sizemode': 'diameter'},
  'name': '0-5',
  'type': 'scattergeo'},
 {'lat': array([34.456, 34.478, 34.476, 34.474, 34.436]),
  'locationmode': 'USA-states',
  'lon': array([-119.371, -119.409, -119.421, -119.431, -119.351]),
  'marker': {'color': 'rgb(255,128,0)',
   'line': {'color': 'rgb(40,40,40)', 'width': 0.5},
   'size': 15.005,
   'sizemode': 'diameter'},
  'name': '5-10',
  'type': 'scattergeo'},
 {'lat': array([36.255, 34.339, 37.681, 37.212, 36.693]),
  'locationmode': 'USA-states',
  'lon': array([-118.412, -118.32 , -119.592, -119.418, -119.7  ]),
  'marker': {'color': 'rgb(255,255,0)',
   'line': {'color': 'rgb(40,40,40)', 'width': 0.5},
   'size': 15.0,
   'sizemode': 'diameter'},
  'name': '10-15',
  'type'

Next, we will dictate what our figure layout should be. 

In [14]:
layout=dict(
        title='Range of Wildfire Brightnesses in California from 09/01/2017 to 09/01/2018',
        showlegend=True,
        geo = dict(
            projection=dict( type='albers usa'), #provides the gray USA map 
            center=dict(lon=-116.4194, lat=34.9592), #centers the map on the middle of SoCal when you first create the map
            zoom=6, #Zoom factor of map when you create it 
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )
fig = dict(data=fires, layout=layout) #our figure with fires data and the layout we want 
py.iplot(fig, validate=False, filename='SoCal-FIRMS-bubble-map') #plot the data! 

<img src = "../images/firstplot.PNG">

## Add more data points! 
Here, I am just doing the same steps but getting a total of 300 data points in the end and tweaking the map.

In [15]:
brightest100=py_data.sort('brightness', descending=True).take(range(0,100))
brightest100

latitude,longitude,brightness,acq_date
34.466,-119.078,504,2017-12-06
34.47,-119.652,503.7,2017-12-16
34.549,-119.35,502.4,2017-12-11
34.415,-119.41,502.3,2017-12-10
34.477,-119.079,501.9,2017-12-06
34.474,-119.096,501.5,2017-12-06
34.444,-119.334,501.1,2017-12-09
34.48,-119.654,500.1,2017-12-16
34.344,-119.185,500,2017-12-05
34.404,-119.146,500,2017-12-05


In [16]:
dimmest100=py_data.sort('brightness', descending=False).take(range(0, 100))
dimmest100

latitude,longitude,brightness,acq_date
36.255,-118.412,300,2017-10-16
34.339,-118.32,300,2017-12-06
37.681,-119.592,300,2017-10-08
37.212,-119.418,300,2017-12-06
36.693,-119.7,300,2018-01-26
36.971,-119.957,300,2017-12-04
36.082,-118.757,300,2017-09-11
37.243,-120.353,300,2018-01-26
36.546,-119.693,300.1,2017-12-20
34.461,-118.964,300.1,2017-12-12


In [17]:
bright_col=py_data.sort('brightness',descending=False).column('brightness')#the brightness column of our data
medium=np.median(bright_col) #find median of brightnesses
list(bright_col).index(medium) #find index of median


3017

In [18]:
middle100=py_data.take(range((3017-50),(3017+50))) #take two above and two below median to get "middle 100 brightest"
middle100

latitude,longitude,brightness,acq_date
34.389,-119.176,378.7,2017-12-05
34.415,-119.231,390,2017-12-05
34.404,-119.231,382.6,2017-12-05
34.393,-119.234,427.3,2017-12-05
34.362,-119.276,344.9,2017-12-05
34.416,-119.245,351.1,2017-12-05
34.339,-119.299,340.9,2017-12-06
34.478,-119.164,434.8,2017-12-06
34.373,-119.407,373.8,2017-12-06
34.364,-119.359,349.9,2017-12-06


In [19]:
lats300=np.append(np.append(brightest100.column(0),(middle100.column(0))), dimmest100.column(0))
longs300=np.append(np.append(brightest100.column(1),(middle100.column(1))), dimmest100.column(1))
brights300=np.append(np.append(brightest100.column(2),(middle100.column(2))), dimmest100.column(2))
dates300=np.append(np.append(brightest100.column(3),(middle100.column(3))), dimmest100.column(3))

In [20]:
filtered_data300=Table().with_columns(
        'latitude', lats300,
        'longitude', longs300,
        'brightness', brights300,
        'acq_date', dates300
        )

One thing I wanted to do was add the brightness value and date to the label of each bubble. 

First, I created a function called "labels" that takes in two arrays and combines their values into a single array. For example, I took the "brightness" column and "acq_date" column from our table and returned an array with "< brightness value >, < date >" for each fire. 

In [21]:
def labels(array1,array2):
    a=array1.astype(str)
    b=array2.astype(str)
    label=make_array()
    for i in range(0,len(array1)):
        matched_label=a[i]+', '+b[i]
        label=np.append(label, matched_label)
    return label 

In [22]:
labels300=labels(filtered_data300.column('brightness'),filtered_data300.column('acq_date')) #create array of 300 labels 
labels300[0] #example of a single label 

'504.0, 2017-12-06'

Then, I added a "text" attribute to the fire dictionary and set it equal to a specific splice of the "labels300" array. I also changed the trace names in the legend by creating a new list of labels called "legend labels" and editing the "name" attribute in the fire dictionary.

In [23]:
limits300=[(0,100),(100,200),(200,300)]
legendlabels=['Bright','Moderate', 'Dim'] #new legend labels 
fires300=[] 
for i in range(len(limits)): 
    group=filtered_data300.take(range(limits300[i][0], limits300[i][1])) 
    fire=dict(
        type = 'scattergeo',
        locationmode = 'USA-states', 
        lon = group['longitude'],
        lat = group['latitude'],
        text= labels300[range(limits300[i][0], limits300[i][1])],
        location=['California'], 
        marker=dict(
                size=group['brightness']/20, 
                color=colors[i], 
                line = dict(width=0.5, color='rgb(40,40,40)'), 
                sizemode='diameter'
                ), 
        name=legendlabels[i]) 
    fires300.append(fire)


In [24]:
layout300=dict(
        title='Range of Wildfire Brightnesses in California from 09/01/2017 to 09/01/2018',
        showlegend=True,
        opacity=0.7, #made the bubbles see-through 
        geo = dict(
            projection=dict( type='albers usa'), 
            center=dict(lon=-119.4179, lat=36.7783), 
            showland = True,
            landcolor = 'rgb(64,64,64)', #changed color of map to dark gray 
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )
fig = dict(data=fires300, layout=layout300)
py.iplot(fig, validate=False, filename='FIRMS300-bubble-map') 

<img src = "../images/secondplot.PNG">

Tip: you can hide a specific trace by clicking on its bubble in the legend! 

As we can see, the most intense fires of 2017 seem to be located in the area north of Los Angeles. The bubble labels indicate that these fires spanned the first two weeks of December 2017. On the other hand, fires with less intensity were scattered up and down the Central Valley. 

## Using the CA Oct 2017- Apr 2018 dataset

The `ca1718.csv` file contians FIRMS data for Calfornia's 2017-2018 fire season, which lasts from October to April. I performed the same actions as above, taking a total of 300 data points for the visualization.

In [84]:
data1718=Table().read_table('~/URAP/google_earth_fires/data/ca1718.csv').sort('bright_ti4', descending=True).select(0,1,2,5)
data1718

latitude,longitude,bright_ti4,acq_date
39.3832,-121.929,367,2018-02-13
39.3774,-121.928,367,2018-02-13
39.2687,-121.866,367,2018-02-12
39.3549,-121.974,367,2017-10-06
39.3523,-121.968,367,2017-10-06
39.3514,-121.973,367,2017-10-05
39.8693,-122.057,367,2018-02-14
39.8702,-122.063,367,2018-02-13
39.8704,-122.058,367,2018-02-13
39.8735,-122.061,367,2018-02-13


In [85]:
bright1718=data1718.take(range(0,100))
dim1718=data1718.sort('bright_ti4', descending=False).take(range(0,100))
bright_col1718=data1718.sort('bright_ti4',descending=False).column('bright_ti4')#the brightness column of our data
med1718=np.median(bright_col1718) #find median of brightnesses
list(bright_col1718).index(med1718) #find index of median

16285

In [86]:
mid1718=data1718.take(range((16285-50),(16285+50))) #take two above and two below median to get "middle 50 brightest"
mid1718

latitude,longitude,bright_ti4,acq_date
38.3503,-122.323,331.4,2018-04-03
38.4058,-122.319,331.4,2017-10-09
38.4127,-122.278,331.4,2017-10-09
34.3841,-119.431,331.4,2017-12-08
36.983,-120.42,331.4,2018-02-15
37.1733,-120.34,331.4,2017-11-28
36.8601,-120.288,331.4,2018-01-19
39.1788,-120.592,331.4,2017-11-07
39.362,-121.355,331.4,2017-10-09
39.4163,-121.358,331.4,2017-10-09


In [87]:
lats1718=np.append(np.append(bright1718.column(0),(mid1718.column(0))), dim1718.column(0))
longs1718=np.append(np.append(bright1718.column(1),(mid1718.column(1))), dim1718.column(1))
brights1718=np.append(np.append(bright1718.column(2),(mid1718.column(2))), dim1718.column(2))
dates1718=np.append(np.append(bright1718.column(3),(mid1718.column(3))), dim1718.column(3))

In [88]:
filtered1718=Table().with_columns(
        'latitude', lats1718,
        'longitude', longs1718,
        'brightness', brights1718,
        'acq_date', dates1718
        )

In [90]:
labels1718=labels(filtered1718.column('brightness'),filtered1718.column('acq_date'))

In [95]:
limits1718=[(0,100),(100,200),(200,300)]
legendlabels=['Bright','Moderate', 'Dim']
fires1718=[] 
for i in range(len(limits1718)): 
    group=filtered1718.take(range(limits1718[i][0], limits1718[i][1])) #i.e: in the first iteration, we are taking the top 100 rows (the brightest) 
    fire=dict(
        type = 'scattergeo',
        locationmode = 'USA-states', 
        lon = group['longitude'],
        lat = group['latitude'],
        text= labels1718[range(limits1718[i][0], limits1718[i][1])], 
        marker=dict(
                size=group['brightness']/20, 
                color=colors[i], 
                line = dict(width=0.5, color='rgb(40,40,40)'), #the outline of each bubble 
                sizemode='diameter'
                ), 
        name=legendlabels[i]) 
    fires1718.append(fire)

In [96]:
layout1718=dict(
        title='Range of Wildfire Brightnesses in California from 10/01/2017 to 04/30/2018',
        showlegend=True,
        opacity=0.7,
        geo = dict(
            projection=dict( type='albers usa'), 
            center=dict(lon=-119.4179, lat=36.7783), 
            showland = True,
            landcolor = 'rgb(64,64,64)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )
fig = dict(data=fires1718, layout=layout1718)
py.iplot(fig, validate=False, filename='FIRMS1718-bubble-map') 

<img src = "../images/thirdplot.PNG">

# Conclusion 
It was quite an experience learning how to translate data that I found on Google Earth Engine into a visual representation using Plotly. Plotly was quite difficult to figure out at first, but I was able to make a few tweaks that improved the graph a bit. It was interesting to see how a majority of wildfires occurred around Central California. Additionally, the majority of the worst fires during the 2017-18 wildfire season occurred in Northern California. 

There are still a few things I'd like to tweak in this project. I spent a while trying to figure out how to get a gradient scale in the legend to see just how much brightness varies among these fires, but was unable to find a way. There are also multiple data points that represent the same fires, except the data were recorded on different days. It would be helpful to clean the data a bit more to ensure there are no repetitive points. This would also clean up the visual aspect of the map. 

Overall, I was glad to have learned more about data visualization and the Plotly Python Library. Familiarizing myself with all the methods will definitely be helpful for future data exploration. I was also surprised by the amount of fire data there are on the internet, and I'm glad that Google Earth Engine provides an easy way to discover such datasets. 